In [1]:
from numpy.random import seed
seed(13)
from tensorflow import set_random_seed
set_random_seed(13)

from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, MaxPooling3D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras.models import model_from_json
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from operator import itemgetter
import numpy as np
import os
import cv2             
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
import shutil
import os

Using TensorFlow backend.


# Load Data

In [ ]:
train = pd.read_csv(r'datasets\test.txt', sep=" ")

In [4]:
# %load conll_dictorizer.py

[{'id': '-DOCSTART- -X- -X- O'}]
{'id': '-DOCSTART- -X- -X- O'}
<class '__main__.Token'>
[{'id': 'EU NNP B-NP B-ORG'}, {'id': 'rejects VBZ B-VP O'}, {'id': 'German JJ B-NP B-MISC'}, {'id': 'call NN I-NP O'}, {'id': 'to TO B-VP O'}, {'id': 'boycott VB I-VP O'}, {'id': 'British JJ B-NP B-MISC'}, {'id': 'lamb NN I-NP O'}, {'id': '. . O O'}]
La
True
{'cpos', 'pos', 'feats', 'form', 'id', 'lemma'}
{'pos', 'id', 'lemma', 'cpos', 'feats', 'form'}
dict_keys(['id', 'form', 'lemma', 'cpos', 'pos', 'feats'])
['el', 'La', '1', 'num=s|gen=f', 'da', 'd']
['el', 'La', '1', 'num=s|gen=f', 'da', 'd']
['el', 'La', '1', 'num=s|gen=f', 'da', 'd']
Token value: dict_values(['1', 'La', 'el', 'd', 'da', 'num=s|gen=f'])
['d', 'el', 'La', '1', 'num=s|gen=f', 'da']


In [7]:
# %load datasets.py

[{'form': '-DOCSTART-', 'ppos': '-X-', 'pchunk': '-X-', 'ner': 'O'}]
[{'form': 'EU', 'ppos': 'NNP', 'pchunk': 'B-NP', 'ner': 'B-ORG'}, {'form': 'rejects', 'ppos': 'VBZ', 'pchunk': 'B-VP', 'ner': 'O'}, {'form': 'German', 'ppos': 'JJ', 'pchunk': 'B-NP', 'ner': 'B-MISC'}, {'form': 'call', 'ppos': 'NN', 'pchunk': 'I-NP', 'ner': 'O'}, {'form': 'to', 'ppos': 'TO', 'pchunk': 'B-VP', 'ner': 'O'}, {'form': 'boycott', 'ppos': 'VB', 'pchunk': 'I-VP', 'ner': 'O'}, {'form': 'British', 'ppos': 'JJ', 'pchunk': 'B-NP', 'ner': 'B-MISC'}, {'form': 'lamb', 'ppos': 'NN', 'pchunk': 'I-NP', 'ner': 'O'}, {'form': '.', 'ppos': '.', 'pchunk': 'O', 'ner': 'O'}]


In [ ]:
train_dict[:10]

# Data preprocessing

In [41]:
# train_dict is a list of lists of dictionaries
def extract_WORDS_NER_TAGS(train_dict):
    X, Y = [], []
    
    for list_dict in train_dict: # list of dicts
        for d in list_dict:
            word = d['form'].lower()
            ner = d['ner'].lower()
            if word != "-docstart-":
                X.append(word)
                Y.append(ner)
    
    return X, Y

In [54]:
def create_indices_words(X, glove):
    d_indices = {}
    # 0 = padding,  1 = unknown
    c = 2
    
    # Add all glove words
    for word in glove:
        if word not in d_indices:
            d_indices[word] = c
            c += 1
            
    # Add all words from training set       
    for word in X:
        if word not in d_indices:
            d_indices[word] = c
            c += 1
        
    return d_indices

In [55]:
def create_indices_NER(Y):
    d_indices = {}
    c = 0
    
    # Add all NER tags from training set       
    for tag in Y:
        if tag not in d_indices:
            d_indices[tag] = c
            c += 1
        
    return d_indices

In [56]:
def create_inverted_indices(d_indices):
    return {v: k for k, v in d_indices.items()}

In [57]:
def load_GloVe(file):
    embeddings_dict = {}
    glove = open(file, encoding='utf-8')
    
    for line in glove:
        line = line.strip().split()
        word = line[0]
        embedding_vec_word = np.array(line[1:], dtype='float32')
        embeddings_dict[word] = embedding_vec_word
        
    glove.close()
    return embeddings_dict

#### Extract WORDS and NER Tags

In [47]:
X, Y = extract_WORDS_NER_TAGS(train_dict)

#### Load GloVe vector embeddings 

In [51]:
embeddings_dict = load_GloVe('glove.6B/glove.6B.100d.txt')

#### Create indices and inverted indices

In [58]:
X_indices = create_indices_words(X, embeddings_dict)
Y_indices = create_indices_NER(Y)

In [59]:
X_inv_indicies = create_inverted_indices(X_indices)
Y_inv_indicies = create_inverted_indices(Y_indices)

#### Calculate most similar word embeddings 

In [60]:
def most_similar_embeddings(embeddings_dict, key_word, n):
    
    cs_dict = {}
    key_word_embedding = embeddings_dict[key_word]
    
    for word, embedding in embeddings_dict.items():
        cs = cosine_similarity([key_word_embedding], [embedding])[0][0]
        cs_dict[word] = cs
    
    cs_sorted = sorted(cs_dict.items(), key=itemgetter(1), reverse=True)[1:n+1] # Do not return table
        
    return cs_sorted

In [61]:
cs_most_similar = most_similar_embeddings(embeddings_dict, 'table', 5)

In [62]:
cs_most_similar

[('tables', 0.8021162),
 ('place', 0.6582378),
 ('bottom', 0.65597194),
 ('room', 0.6543691),
 ('side', 0.6433667)]

#### Create GloVe word embedding matrix

In [63]:
def fill_GloVe_matrix(X_indices, embeddings_dict):
    c = 0
    for word in X_indices:
        embedding = embeddings_dict.get(word)
        if embedding is not None:
            word_embedding_matrix[c] = embedding
        else:
            word_embedding_matrix[c] = np.random.rand(1, 100)
        c += 1
    return word_embedding_matrix

In [64]:
word_embedding_matrix = np.random.rand(len(X_indices), 100)
word_embedding_matrix = fill_GloVe_matrix(X_indices, embeddings_dict)

#### Create sequenced data

In [171]:
def build_sequences(X, Y, X_indices, Y_indices):
    # Find sequences. Each sequence is seperated by '.'
    X_all_seq = []
    X_seq = []
    Y_all_seq = []
    Y_seq = []
    
    for i, w in enumerate(X):
        X_seq.append(X_indices.get(w))
        Y_seq.append(Y_indices.get(Y[i]))

        if w == '.':
            X_all_seq.append(X_seq)
            X_seq = []
            Y_all_seq.append(Y_seq)
            Y_seq = []

    max_len = max(len(l) for l in X_all_seq)
        
    X_all_seq = pad_sequences(X_all_seq, max_len)
    Y_all_seq = pad_sequences(Y_all_seq, max_len)
    
    return X_all_seq, Y_all_seq

In [172]:
X_seq, Y_seq = build_sequences(X, Y, X_indices, Y_indices)